In [22]:
from deeprhythm import DeepRhythmPredictor
import torchaudio
import librosa

model = DeepRhythmPredictor()
waveform, sr = torchaudio.load('/disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test/Track01876/mix.flac')

waveform = waveform.view(-1)
tempo = model.predict_from_audio(waveform, sr)

# to include confidence
tempo, confidence = model.predict_from_audio(waveform, sr, include_confidence=True)

print(f"Predicted Tempo: {tempo} BPM")

Model weights already exist.
Predicted Tempo: 121.0 BPM


/home/demancum/miniconda3/envs/cocola_test/lib/python3.11/site-packages/deeprhythm/utils.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  clip_tensor = torch.tensor(audio[i:i + clip_samples], dtype=torch.float32)


In [ ]:
import sys
import essentia
from essentia.streaming import *


# initialize algorithms we will use
loader = MonoLoader(filename='/disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test/Track01876/mix.flac')
framecutter = FrameCutter()
windowing = Windowing(type="blackmanharris62")
spectrum = Spectrum()
spectralpeaks = SpectralPeaks(orderBy="magnitude",
                              magnitudeThreshold=1e-05,
                              minFrequency=40,
                              maxFrequency=5000, 
                              maxPeaks=10000)
hpcp = HPCP()
key = Key()

# use pool to store data
pool = essentia.Pool() 

# connect algorithms together
loader.audio >> framecutter.signal
framecutter.frame >> windowing.frame >> spectrum.frame
spectrum.spectrum >> spectralpeaks.spectrum
spectralpeaks.magnitudes >> hpcp.magnitudes
spectralpeaks.frequencies >> hpcp.frequencies
hpcp.hpcp >> key.pcp
key.key >> (pool, 'tonal.key_key')
key.scale >> (pool, 'tonal.key_scale')
key.strength >> (pool, 'tonal.key_strength')

# network is ready, run it
essentia.run(loader)

print(pool['tonal.key_key'] + " " + pool['tonal.key_scale'])




F# major
